In [1]:
path="C:/Users/dexte/Documents/GitHub/Quadruped/Quadruped_sim/urdf/"
path="/its/home/drs25/Documents/GitHub/Quadruped/Quadruped_sim/urdf/"

import os
import numpy as np
from environment import environment
from environment import p
from agent import agent
clear = lambda: os.system('clear')
import time
def update(env,imer=1000,show=0):
    for k in range(imer): #update simulation
        p.stepSimulation()
        if not show:
            p.setTimeStep(1./240.)
        else:
            time.sleep(1./240.)

pybullet build time: Nov 28 2023 23:45:17


# parameters

In [2]:
# Initialize the PyBullet physics engine
env=environment(True)
a=agent()
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))
env.runTrial(a,100)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=Quadro P620/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.183.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.183.01
Vendor = NVIDIA Corporation
Renderer = Quadro P620/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


0

# manual

In [3]:
env.reset()
update(env)

In [4]:
def safe_get(i):
     return env.quad.motors[i]
def safe_set(i,angle):
     a=env.quad.motors.copy()
     a[i]=angle
     env.quad.setPositions(a)
#env.quad.setPositions(np.array([50 for i in range(12)]))
def schedule_move(target_angles, step_size=1):
        # Flag to track if any servo still needs to move
        moving = True

        while moving:
            moving = False  # Assume all servos have reached their target for this iteration

            for i, target_angle in enumerate(target_angles):
                #print(i)
                current_angle = safe_get(i) or 0 # Handle case where angle is None
                # Calculate step direction based on difference
                if not(current_angle>target_angle-1 and current_angle<target_angle+1): #within bounds
                    if current_angle < target_angle:
                        new_angle = min(current_angle + step_size, target_angle)
                        moving = True
                    elif current_angle > target_angle:
                        new_angle = max(current_angle - step_size, target_angle)
                        moving = True
                    else:
                        new_angle = current_angle  # Servo is already at target

                    # Set the new angle
                    safe_set(i,new_angle)
            update(env,imer=10,show=True)
            
def crawl_step():
        leg_seq=[[30,-30,0],[0,30,0]] #,[-30,0,0]
        bod_seq=[0,2,1,3]
        
        for j in range(len(bod_seq)):
            leg=bod_seq[j]
            for i in range(len(leg_seq)):
                if leg==0 or leg==2:
                    leg_seq[i][0]*=-1
                    leg_seq[i][1]*=-1
                motors=np.array([0 for k in range(3*leg)]+leg_seq[i]+[0 for k in range(12-len([0 for p in range(3*leg)]+leg_seq[i]))])
                seq=motors+np.array([safe_get(k) for k in range(12)])
                seq[seq<0]=0
                seq[seq>180]=180
                schedule_move(seq,5)

        schedule_move(env.quad.neutral,5)        
        
        update(env,imer=100,show=True)
env.reset()
update(env)
for i in range(10):
    crawl_step()